In [8]:
pip install keras-tuner

     |████████████████████████████████| 71kB 3.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=056555a4dbade8cb74215ad5d362cbf344bec22f288c71d36ef6aff16ed07bce
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=391c428f16e2ae272ff1c70f4c6985634514ae142e42872adb4503c6afc20ac8
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [11]:
fashion_mnist=keras.datasets.fashion_mnist

In [12]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [13]:
train_images=train_images/255.0
test_images=test_images/255.0

In [14]:
train_images[0].shape
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [19]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
                                kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                                activation = 'relu', input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                                                kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                                activation = 'relu'),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units',min_value=32,max_value=128,step = 16),activation='relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')

  ])
  model.compile(optimizer=keras.optimizers.Adam(
      hp.Choice('learning_rate',values=[1e-2, 1e-3])
  ),
  loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model
  

In [20]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [24]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy', max_trials=5, directory='output', project_name = "Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels, epochs = 3, validation_split=0.1)


Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |32                
conv_1_kernel     |5                 |3                 
conv_2_filter     |64                |48                
conv_2_kernel     |3                 |3                 
dense_1_units     |96                |128               
learning_rate     |0.01              |0.001             

Epoch 1/3
 332/1688 [====>.........................] - ETA: 3:18 - loss: 0.7255 - accuracy: 0.7449

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        13872     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3539072   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,554,554
Trainable params: 3,554,554
Non-trainable params: 0
_________________________________________________________________
